In [46]:
import pandas as pd
import math
import numpy as np

## Reading in Data

In [36]:
maindf = pd.read_csv('leagueoflegends2021data.csv')
maindf.drop(columns=['Unnamed: 1'], inplace = True)

In [37]:
def odds_to_percentage(odd):
    if odd <= -100:
        return odd * -1 / (odd * -1 + 100)
    elif odd >= 100:
        return 100 / (100 + odd)
    return 0

In [38]:
maindf['Percent1'] = maindf['Odds1'].apply(odds_to_percentage)
maindf['Percent2'] = maindf['Odds2'].apply(odds_to_percentage)

In [39]:
maindf['True1'] = (maindf['Percent1'] + (1 - maindf['Percent2']))/ 2

In [40]:
maindf['True2'] = 1 - maindf['True1']

In [41]:
maindf['Team1'] = maindf['Team'].apply(lambda x: x.split('-')[0][:-1])
maindf['Team2'] = maindf['Team'].apply(lambda x: x.split('-')[1][1:])

In [42]:
maindf.drop(columns=['Team'], inplace= True)

In [43]:
def rateDiff(probability):
    return math.log(1/probability - 1) * 400 * -1

In [44]:
maindf['RateDiff1'] = maindf['True1'].apply(rateDiff)
maindf['RateDiff2'] = maindf['True2'].apply(rateDiff)

## Generate Least-Squared Matrix for Team ELO

In [56]:
index_dict = {key:value for value, key in enumerate(maindf['Team1'].unique())}


In [58]:
bodylist = []
solutionlist = []
for index, row in maindf.iterrows():
    solutionlist.append(row['RateDiff1'])
    alpha = [0 for _ in range(10)]
    alpha[index_dict[row['Team1']]] += 1
    alpha[index_dict[row['Team2']]] -= 1
    bodylist.append(alpha)
bodylist.append([.1 for _ in range(len(index_dict.keys()))])
solutionlist.append(1200)

In [61]:
A = np.array(bodylist)
b = np.array(solutionlist)

In [66]:
first = np.matmul(A.T, A)
pseudo = np.linalg.inv(first)
final = np.matmul(pseudo, np.matmul(A.T, b))

In [67]:
final

array([1072.39775096, 1242.37929605, 1394.97703637, 1014.91119694,
       1482.52393745, 1004.49681137, 1529.26390024,  843.79737814,
       1053.9792207 , 1361.27347178])

In [68]:
index_dict

{'Dignitas': 0,
 'Evil Geniuses': 1,
 '100 Thieves': 2,
 'FlyQuest eSports': 3,
 'Team Liquid': 4,
 'Counter Logic Gaming': 5,
 'Cloud9': 6,
 'Golden Guardians': 7,
 'Immortals': 8,
 'Team SoloMid': 9}